# Obtaining Data

## Stakeholders 
The Seattle Police Department has reached out to determine the likelihood of Terry Stops preventing a crime by arresting suspects under the basis of "probable cause". In an effort to improve race relations, they also want to see how much of a role racial discrimination plays in who gets stopped.  

## Objectives: 
* Determine whether Terry Stops are preventing crimes
* Determine if there is a relationship between Terry Stops and a subject's race
* Do the differences in races of the police officer and the subject play a role?
* 

Ok, let's check out this data!

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('CSV_Files/Terry_Stops.csv')
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,-,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,-,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,...,07:59:00,-,-,-,NaN,N,N,-,-,-
2,-,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,...,19:12:00,-,-,-,NaN,N,-,-,-,-
3,-,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,-,-,-,NaN,N,N,-,-,-
4,-,-1,20150000001739,33155,Field Contact,None,6973,1977,M,White,...,00:41:00,-,-,-,NaN,N,N,-,-,-


That's... A lot of missing data...

In [3]:
df.shape

(43512, 23)

We see a lot of place holder values, let's look for any NaNs in our dataset.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43512 entries, 0 to 43511
Data columns (total 23 columns):
Subject Age Group           43512 non-null object
Subject ID                  43512 non-null int64
GO / SC Num                 43512 non-null int64
Terry Stop ID               43512 non-null int64
Stop Resolution             43512 non-null object
Weapon Type                 43512 non-null object
Officer ID                  43512 non-null object
Officer YOB                 43512 non-null int64
Officer Gender              43512 non-null object
Officer Race                43512 non-null object
Subject Perceived Race      43512 non-null object
Subject Perceived Gender    43512 non-null object
Reported Date               43512 non-null object
Reported Time               43512 non-null object
Initial Call Type           43512 non-null object
Final Call Type             43512 non-null object
Call Type                   43512 non-null object
Officer Squad               42968 non-null ob

Officer Squad has 544 NaN values.  To save time we'll make a function that shows us the value counts of each column

In [5]:
def col_values(df):
    """
    For use in Preprocessing and cleaning to find placeholder values
    Input: Data frame
    Output: Counts of unique values for each column
    """
    for col in df.columns:
        print(col)
        print('\n')
        print(df[col].value_counts())
        print('-------------------------------------------------------')
        print('\n')

In [6]:
col_values(df)

Subject Age Group


26 - 35         14420
36 - 45          9153
18 - 25          8875
46 - 55          5599
56 and Above     2185
1 - 17           1908
-                1372
Name: Subject Age Group, dtype: int64
-------------------------------------------------------


Subject ID


-1              34676
 7726859935        18
 7727117712        12
 7753260438        11
 7726318196         8
                ...  
 7730602336         1
 7758922092         1
 7727677812         1
 9640739188         1
 12095084261        1
Name: Subject ID, Length: 7045, dtype: int64
-------------------------------------------------------


GO / SC Num


20150000190790    16
20160000378750    16
20180000134604    14
20170000132836    13
20190000441736    13
                  ..
20190000076830     1
20150000379928     1
20180000002079     1
20180000333849     1
20180000071981     1
Name: GO / SC Num, Length: 33910, dtype: int64
-------------------------------------------------------


Terry Stop ID


130800

### Notes:
**Age Groups**: 1372 people with unknown age

**Subject ID**: 34,676 people that are not assigned an ID number, also some people stopped multiple times

**General Offense**: Repeated values, need to look into why

**Terry Stop ID**: Repeated values. These should definitely be removed.

**Stop Resolution**: No NaNs or place holders, but distinction needs to be made between physical and non-custodial arrest.

**Weapon Type**: Needs to be cleaned up and binned. Need to create 'unknown' to account for placeholders

**Officer ID**: Clean. This data may be irrelevant.

**Officer YOB**: Clean.

**Officer Gender**: 11 counts of 'N'.  Unable to determine if this means unknown, non-gender binary, etc.  It's a tiny amount of data, just drop it.

**Reported Date**: Clean; but delete the Time counter and extract the year to feature engineer the age of the officer.

**Reported Time**: Clean

**Initial Call/Final/Call Type**: The consistent appearance of 12,828 seems that these were calls made **BEFORE** the integration of the **Computer Assisted Dispatch** system. Although this is a lot of data to just drop, it could cause problems with interpretation since we can't tell if this was a result of an officer being proactive or responding to a call.

**Officer Squad**: We know from earlier that their are NaN values present in this column.

**Arrest Flag**: Clean.  This is our Target because it denotes all Physical Arrests.

**Frisk Flag**: 478 place holders. Delete to improve interpretability. 

**Precinct/Sector/Beat**: Besides the '-' placeholder, there are also place holders for 99 and OOJ.  Also, 15 FK_ERRORS. 

For my sanity and ease of use, we'll quickly change the column names to be more "python friendly".

In [7]:
df.columns = ['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'officer_squad',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat']
df.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'officer_squad',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat'],
      dtype='object')

# Scrubbing

Alright, let's get to scrubbing data! We'll start with the easy stuff and move up to the more difficult decisions.

## Dashes
Until we get into the modeling stage, we'll replace all '-' values with the string 'Unknown'

In [8]:
df = df.replace('-', 'Unknown')

In [9]:
df.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,officer_squad,arrest_flag,frisk_flag,precinct,sector,beat
0,Unknown,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,Unknown,Unknown,Unknown,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,Unknown,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,...,07:59:00,Unknown,Unknown,Unknown,NaN,N,N,Unknown,Unknown,Unknown
2,Unknown,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,...,19:12:00,Unknown,Unknown,Unknown,NaN,N,Unknown,Unknown,Unknown,Unknown
3,Unknown,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,Unknown,Unknown,Unknown,NaN,N,N,Unknown,Unknown,Unknown
4,Unknown,-1,20150000001739,33155,Field Contact,None,6973,1977,M,White,...,00:41:00,Unknown,Unknown,Unknown,NaN,N,N,Unknown,Unknown,Unknown


## Officer Gender

In [10]:
df.officer_gender.value_counts()

M    38584
F     4917
N       11
Name: officer_gender, dtype: int64

As we've already established, we don't know if 'N' stands for 'Not Available', 'Not Disclosed', or even 'Non-Gender Binary'. Since it's such a small amount of data, we'll just drop it.

In [11]:
df = df[df['officer_gender'] != 'N']

df.officer_gender.value_counts()

M    38584
F     4917
Name: officer_gender, dtype: int64

## Officer Squad
Addressing the NaN values

In [12]:
df.officer_squad.isna().value_counts()

False    42968
True       533
Name: officer_squad, dtype: int64

In the interest of preserving as much data as possible, it might be in our best interest to drop this column. What squad an officer belongs to may not matter as much as the demographics of the police officer. We'll keep a copy of the original data frame just in case.

In [13]:
adf = df.copy()
adf = adf.drop('officer_squad', axis=1)
adf.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'arrest_flag',
       'frisk_flag', 'precinct', 'sector', 'beat'],
      dtype='object')

## Subject ID
The identity might affect the model if people arrested were repeat offenders

In [18]:
# subsetting data so analyze the duplicate Subject ID's

ids = adf[adf['subject_id'] > 3]
ids.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_date,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat
1147,Unknown,11621995853,20190000404915,11622044618,Arrest,Unknown,7638,1989,M,White,...,2019-10-30T00:00:00,22:51:57,ARSON - IP/JO,--PROPERTY DEST (DAMG),911,Y,N,West,D,D2
1169,Unknown,7726713382,20200000109117,12800165868,Offense Report,Unknown,7808,1986,M,White,...,2020-03-31T00:00:00,11:45:40,SHOPLIFT - THEFT,--THEFT - SHOPLIFT,911,N,N,Unknown,Unknown,Unknown
1170,Unknown,7727213211,20190000287593,9587320855,Field Contact,Unknown,8649,1991,M,Hispanic or Latino,...,2019-08-04T00:00:00,13:22:42,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,N,North,J,J2
1171,Unknown,7727213211,20190000287593,9587334107,Field Contact,Unknown,8649,1991,M,Hispanic or Latino,...,2019-08-04T00:00:00,13:45:33,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,N,North,J,J2
1172,Unknown,7727213211,20190000367834,10558772346,Field Contact,Unknown,8459,1990,M,Hispanic or Latino,...,2019-10-02T00:00:00,12:39:08,"WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)",--DISTURBANCE - OTHER,911,N,N,West,D,D2


In [17]:
ids.shape

(8830, 22)

In [20]:
ids = ids[ids['arrest_flag'] == 'Y']
ids.shape

(2282, 22)

In [24]:
ids = ids[ids['subject_id'] == 7727430767]

In [25]:
ids

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_date,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat
24142,26 - 35,7727430767,20190000305890,9730643588,Arrest,Unknown,7782,1986,M,White,...,2019-08-17T00:00:00,15:52:35,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),--THEFT - CAR PROWL,911,Y,N,East,G,G2
24143,26 - 35,7727430767,20190000369340,10569538287,Arrest,Unknown,8491,1974,M,White,...,2019-10-03T00:00:00,15:06:56,Unknown,Unknown,Unknown,Y,Y,West,M,M3
24144,26 - 35,7727430767,20190000405808,11636833403,Arrest,Unknown,8389,1987,M,White,...,2019-10-31T00:00:00,17:51:36,SUSPICIOUS STOP - OFFICER INITIATED ONVIEW,--WARRANT SERVICES - FELONY,ONVIEW,Y,Y,Unknown,Unknown,Unknown
24146,26 - 35,7727430767,20200000149247,13111480477,Arrest,Unknown,8394,1991,M,White,...,2020-05-06T00:00:00,00:41:30,TRESPASS,--PROWLER - TRESPASS,911,Y,Y,North,U,U2
24147,26 - 35,7727430767,20200000180687,13305975975,Arrest,Unknown,8556,1995,M,White,...,2020-06-03T00:00:00,18:42:08,"DISTURBANCE, MISCELLANEOUS/OTHER","--ASSAULTS, OTHER",911,Y,N,West,D,D1
24149,26 - 35,7727430767,20200000215252,13871417277,Arrest,Knife/Cutting/Stabbing Instrument,8586,1990,M,White,...,2020-07-17T00:00:00,22:03:04,ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS),--DV - DOMESTIC VIOL/ASLT (ARREST MANDATORY),911,Y,Y,South,R,R3


Because these incidents have separate Terry Stop IDs, Dates, and different reporting officers, these are separate incidents and shouldn't effect our data.

Let's get rid of those -1's and put 'unassigned' in their place.

In [26]:
adf['subject_id'] = adf.subject_id.replace(-1, 'unassigned')
adf.subject_id.value_counts()

unassigned     34671
7726859935        18
7727117712        12
7753260438        11
7726318196         8
               ...  
8335944118         1
13499591239        1
7727655354         1
7745743294         1
12618563586        1
Name: subject_id, Length: 7039, dtype: int64

## General Offense/Street Check Number
Looking into why there are repeated values 

In [29]:
stop_chk = adf[adf['go_sc_num'] > 1]
stop_chk['go_sc_num'].value_counts()

20160000378750    16
20150000190790    16
20180000134604    14
20190000441736    13
20170000132836    13
                  ..
20180000333849     1
20170000130073     1
20200000020505     1
20170000189693     1
20180000071981     1
Name: go_sc_num, Length: 33908, dtype: int64

In [32]:
stop_chk = stop_chk[stop_chk['go_sc_num'] == 20160000378750]
stop_chk

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_date,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat
6521,18 - 25,unassigned,20160000378750,208302,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:31:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
6522,18 - 25,unassigned,20160000378750,208311,Arrest,None,7492,1983,M,White,...,2016-11-02T00:00:00,23:04:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
16465,26 - 35,unassigned,20160000378750,208300,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:22:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
16466,26 - 35,unassigned,20160000378750,208301,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:24:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
16467,26 - 35,unassigned,20160000378750,208303,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:35:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
16468,26 - 35,unassigned,20160000378750,208307,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:48:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
29312,36 - 45,unassigned,20160000378750,208299,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:18:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
29313,36 - 45,unassigned,20160000378750,208305,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:41:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
29314,36 - 45,unassigned,20160000378750,208308,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:51:00,Unknown,Unknown,Unknown,N,Y,North,N,N3
29315,36 - 45,unassigned,20160000378750,208310,Offense Report,None,7492,1983,M,White,...,2016-11-02T00:00:00,22:59:00,Unknown,Unknown,Unknown,N,Y,North,N,N3


From the dates, the separate Terry Stop ID's, the different Stop Resolutions and it all roughly happening within the same hour, it appears that this was a dispute of some sort in which an officer collected Offense Reports from 12 people and issued out tickets to 4 people (because there was no physical arrest denoted by the column 'arrest_flag', these were non-custodial arrests/citations).  

## Report Date
We want to remove the time stamp from date, create a column for the year and finally calculate the ages for the officers 